In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np

# 데이터 로드
# CSV 파일에서 데이터 로드
data = pd.read_csv('data.csv')

# 데이터 탐색
print("데이터 샘플:\n", data.head())
print("\n데이터 정보:\n")
data.info()
print("\n기술 통계:\n", data.describe())

# 결측치 확인
print("\n결측치 개수:\n", data.isnull().sum())

# 결측치 시각화
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(data.isnull(), cbar=False)
plt.show()

# 결측치 처리
from sklearn.impute import SimpleImputer

# 수치형 변수와 범주형 변수 구분
numerical_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()

# 수치형 변수 결측치 처리 (평균 대체)
mean_imputer = SimpleImputer(strategy='mean')
data[numerical_cols] = mean_imputer.fit_transform(data[numerical_cols])

# 범주형 변수 결측치 처리 (최빈값 대체)
mode_imputer = SimpleImputer(strategy='most_frequent')
data[categorical_cols] = mode_imputer.fit_transform(data[categorical_cols])

# 범주형 변수 인코딩 (원-핫 인코딩)
data = pd.get_dummies(data, columns=categorical_cols)

# 특징과 레이블 분리
X = data.drop('target', axis=1)
y = data['target']

# 데이터 스케일링
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 데이터 분할 (훈련 세트와 테스트 세트)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 모델 선택 및 학습
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)

# 하이퍼파라미터 튜닝 (옵션)
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# 최적의 모델
best_model = grid_search.best_estimator_

# 예측 수행
y_pred = best_model.predict(X_test)

# 모델 평가
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

accuracy = accuracy_score(y_test, y_pred)
print("정확도:", accuracy)

precision = precision_score(y_test, y_pred, average='weighted')
print("정밀도:", precision)

recall = recall_score(y_test, y_pred, average='weighted')
print("재현율:", recall)

f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 점수:", f1)

# 혼동 행렬
cm = confusion_matrix(y_test, y_pred)
print("혼동 행렬:\n", cm)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('예측값')
plt.ylabel('실제값')
plt.show()

# 분류 보고서
report = classification_report(y_test, y_pred)
print("분류 보고서:\n", report)

# ROC-AUC (이진 분류인 경우)
from sklearn.metrics import roc_auc_score, roc_curve

if len(np.unique(y_test)) == 2:
    y_prob = best_model.predict_proba(X_test)[:, 1]
    auc_score = roc_auc_score(y_test, y_prob)
    print("ROC AUC 점수:", auc_score)

    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC 곡선')
    plt.legend(loc='lower right')
    plt.show()

# 중요 특징 확인
importances = best_model.feature_importances_
feature_names = X.columns
feature_importance = pd.Series(importances, index=feature_names)
print("\n특징 중요도:\n", feature_importance.sort_values(ascending=False))

# 중요 특징 시각화
feature_importance.sort_values(ascending=False).head(10).plot(kind='bar')
plt.title('상위 10개 특징 중요도')
plt.show()